# Packages

In [1]:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sns
import astropy.io.fits   as pf   # Pyfits is depracated
import astropy.constants as ct
import ppxf.miles_util   as lib  # MILES library embedded in ppxf
import ppxf.ppxf_util    as util
import ppxf              as ppxf_module
import os
import scipy.interpolate as sci
from ppxf.ppxf           import ppxf
from ppxf.ppxf_util      import log_rebin

# Config. inputs

In [2]:
seds_list    = np.loadtxt('../../../GAMAII_SPEC/WHAN_NA_UVUP/gal_list.txt', dtype=str)
seds_path    = '../../../GAMAII_SPEC/WHAN_NA_UVUP/'
results_path = './Results/'
ppxf_dir     = os.path.dirname(os.path.realpath(ppxf_module.__file__))
miles_lib    = ppxf_dir+'/miles_models/Mun1.30*.fits'
light_speed  = float(str(ct.c.to('km/s')).split(' ')[0])               # selecting the numerical values (must be in km/s)

In [5]:
miles01_teste = pf.open(ppxf_dir+'/miles_models/Mun1.30Zm0.40T00.0631_iPp0.00_baseFe_linear_FWHM_2.51.fits')

In [10]:
miles01_teste.info()

Filename: /home/mlldantas/anaconda3/envs/py3k6/lib/python3.6/site-packages/ppxf/miles_models/Mun1.30Zm0.40T00.0631_iPp0.00_baseFe_linear_FWHM_2.51.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      63   (4300,)   float32   


In [9]:
miles02_teste = pf.getdata(ppxf_dir+'/miles_models/Mun1.30Zm0.40T00.0631_iPp0.00_baseFe_linear_FWHM_2.51.fits')

# Plot parameters (because I'm lazy)

In [ ]:
palette      = ['#af8dc3','#f7f7f7','#7fbf7b']
transparency = 0.8
lw           = 2.

# pPXF loop

In [ ]:
for each_sed in range(seds_list.size):
    # flux ----------------------------------------------------------------------------------------------------------------
    sed_file  = os.path.join(seds_path, seds_list[each_sed])          # link between path and files
    sed_flux  = pf.getdata(sed_file)[0]                               # opening each spectra
    sed_err   = pf.getdata(sed_file)[1]                               # opening each spectra error
    
    # wavelength ----------------------------------------------------------------------------------------------------------
    sed_header = pf.open(sed_file)[0].header
    sed_wl_ini = sed_header['WMIN'] 
    sed_wl_stp = sed_header['CD1_1']
    sed_wl     = np.arange(sed_flux.size)*(sed_wl_stp)+sed_wl_ini
    
    # cleaning flux and wavelength trash ----------------------------------------------------------------------------------
    idx_clean       = np.where((np.logical_not(np.isnan(sed_flux)))*(sed_err>0))    # indices where NAN is present
    flux_temp       = sed_flux[idx_clean]                                         # removing NAN values from spectra
    flux_temp_err   = sed_err[idx_clean]
    wavelength_temp = sed_wl[idx_clean]
    
    # limiting the spectrum regarding the wavelength range of the library -------------------------------------------------
    wl_lib_idx = (wavelength_temp>3540) & (wavelength_temp<7409)       # MILES wavelength range
    flux       = flux_temp[wl_lib_idx]
    flux_err   = flux_temp_err[wl_lib_idx]
    wavelength = wavelength_temp[wl_lib_idx]
    
    # other important parameters ------------------------------------------------------------------------------------------
    redshift = sed_header['Z']
    cataid   = sed_header['CATAID']
    
    # ppxf ----------------------------------------------------------------------------------------------------------------
    ### parameters --------------------------------------------------------------------------------------------------------
    vel_scale          = light_speed*np.log(wavelength[1]/wavelength[0])            # velocity scale - Eq.8 Capellari 2017
    fwhm_gama          = 2.                                       # MUST BE CHECKED!!!!! This is just a temporary value!!!
    miles              = lib.miles(miles_lib, vel_scale, fwhm_gama)
    stellar_templates  = miles.templates.reshape(miles.templates.shape[0], -1)
    dv                 = light_speed*(miles.log_lam_temp[0] - np.log(wavelength[0]))  # eq.(8) of Cappellari (2017)
    vel                = light_speed*np.log(1 + redshift)                             # eq.(8) of Cappellari (2017)
    start              = [vel, 180.]        
    
    ### rebinning the spectrum so it is at the same size as the templates (or smaller) ------------------------------------
    new_shape          = stellar_templates[:,0].shape
    rebinning_function = sci.interp1d(wavelength, flux)   # for the flux
    wavelength_rebin   = np.linspace(start=wavelength.min(), stop=wavelength.max(), num=int(new_shape[0]))
    flux_rebin         = rebinning_function(wavelength_rebin)
    
    ### rebinning the error -----------------------------------------------------------------------------------------------
    rebin_func_error   = sci.interp1d(wavelength, flux_err)
    wl_rebin_error     = np.linspace(start=wavelength.min(), stop=wavelength.max(), num=int(new_shape[0]))
    flux_err_rebin     = rebin_func_error(wl_rebin_error)
    
    ### final galaxy spectrum and its error to be analysed ----------------------------------------------------------------
    galaxy             = flux_rebin/np.median(flux_rebin)                  # Normalize spectrum to avoid numerical issues
    noise              = flux_err_rebin/np.median(flux_err_rebin)          # idem
 
    
    ### SED FITTING WITH PPXF ---------------------------------------------------------------------------------------------
    sed_fit = ppxf(templates=stellar_templates, galaxy=galaxy, noise=noise, velscale=vel_scale, start=start, plot=False, 
                   lam=wavelength_rebin, moments=4, degree=50)
    
    residues = (galaxy-sed_fit.bestfit)
    
    print (sed_fit.gas_mpoly, sed_fit.gas_reddening, sed_fit.weights, sed_fit.status)
    
    ### Saving plots ------------------------------------------------------------------------------------------------------
    plt.subplots(figsize=(10,5))
    plt.subplot(2,1,1)
    plt.plot(wavelength_rebin, sed_fit.galaxy, '-', color=palette[-1], linewidth=lw, label="Spectrum")
    plt.plot(wavelength_rebin, sed_fit.bestfit, '-', color=palette[0], linewidth=lw, label="Best Fit")
    plt.legend(loc='best')
    plt.xlabel("Wavelength ($\mathrm{\AA}$)", fontsize=12)
    plt.ylabel("Norm. flux", fontsize=12)
    plt.subplot(2,1,2)
    plt.plot(wavelength_rebin, residues, '.', color=palette[-1], linewidth=lw, label="Residues", alpha=transparency)
    plt.axhline(y=0, c='black')
    plt.legend(loc='best')
    plt.xlabel("Wavelength ($\mathrm{\AA}$)", fontsize=12)
    plt.ylabel("Residues", fontsize=12)
    plt.tight_layout()
    plt.savefig(os.path.join('./../../Codes/SEDFitting/Results/', 'gal_'+str(cataid)+'.png'))
    plt.show()